<a href="https://colab.research.google.com/github/Amandaagambire/Galamsey-Watch/blob/main/G_watch_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api geemap folium rasterio
!pip install segmentation-models-pytorch